In [13]:
import pandas as pd
import numpy as np

In [14]:
file = open('beerguide.txt', 'rt')
lines = file.readlines()
file.close()


In [15]:
lines[:] = [element for element in lines if element != '\n' ]
lines = [i.split('\n', 1)[0] for i in lines]

In [16]:
#get the list of all 79 beers
a = lines.index('Table of Contents')
b = lines.index('Specialty Beer')

beerlist = lines[a:b+1]
beerlist[:] = [element for element in beerlist if '\ue004' not in element][1:]



In [17]:
word_replace_dict = {'\xa0': ' ', 
                     '\xad': '-',
                     '\x0c':'',
                    '\ue000':'B',
                    '\ue001':'E',
                    '\ue002':'S',
                    '\ue004':'s',
                    '\ue006':'y'}
beerdes = lines[b+1:]

for k,v in word_replace_dict.items():
    beerdes = [word.replace(k,v) for word in beerdes]

In [18]:
feature = ['IBU', 'BU:GU', 'CO2 Volumes', 'Apparent Attenuation', 'Alcohol', 
             'Collar of Foam (Head Retention)', 'Collar of Foam (Texture)',
            'Hop Aroma/Flavor', 'Hop bitterness','Malt Aroma/Flavor',
            'Palate Body', 'Palate Carbonation', 'Palate Length/Finish', 'Esters']

In [97]:
def beer_info(beer,character):
    l = []
    for i in range(len(character)):
        if character[i] == 'Alcohol':
            l.append([beer [beer.index('Alcohol')+1]][0])
        
        else:
            a = [element for element in beer if character[i] in element]
            if a:
                l.append(a[0].split(character[i],1)[1])
            else:
                l.append(np.nan)
               
    return l

In [98]:
# %debug
beer_dic = {}

for j in range(len(beerlist)):
    if j == len(beerlist)-1:
        b = beerdes[beerdes.index(beerlist[j]):]
    else:
        b = beerdes[beerdes.index(beerlist[j]):beerdes.index(beerlist[j+1])]
    beer_dic[beerlist[j]] = beer_info(beer = b, character = feature)

In [99]:
df = pd.DataFrame(beer_dic, index= feature).transpose()
df.fillna('None', inplace = True)

In [100]:
df.loc['Bohemian-Style Pilsener'].IBU = '30 - 45'
df.loc['Bohemian-Style Pilsener']['Hop bitterness'] = 'medium'

In [101]:
for i in df.columns:
    print(i, '\n', df[i].unique())

IBU 
 [' 25 - 45' ' 18 - 30' ' 60 - 100' ' 50 - 70' ' Varies' ' 10 - 22'
 ' 35 - 50' ' 55 - 85' ' 50 - 80' ' 5 - 15' ' 30 - 50' ' 35 - 60'
 ' 10 - 35' ' 45 - 85' ' 35 - 40' ' 15 - 30' ' 20 - 35' ' 5 - 18'
 ' 15 - 21' ' 20 - 50' ' 9 - 23' ' 20 - 30' ' 25 - 50' ' 20 - 38'
 ' 20 - 45' ' 10 - 17' ' 3 - 6' ' 15 - 25' '30 - 45' ' 40 - 60' ' 35 - 45'
 ' 15 - 40' ' 15 - 45' ' 10 - 15' ' 35 - 63' ' 10 - 24' ' 20 - 40'
 ' 30 - 65' ' 23 - 29' ' 5 - 45' ' 25 - 52' ' 17 - 27' ' 16 - 25'
 ' 18 - 25' ' 18 - 28' ' 25 - 40' ' 22 - 30' ' 15 - 35' ' 5 - 40'
 ' 1 - 100' ' 65 - 100' ' 30 - 40' ' 20 - 28' ' 5 - 70' ' 25 - 35'
 ' 9 - 25' ' 22 - 28']
BU:GU 
 [' 0.62 - 0.78' ' 0.43 - 0.54' ' 0.67 - 0.83' ' 0.89 - 0.93' ' Varies'
 ' 0.62 - 0.75' ' 0.23 - 0.42' ' 0.83 - 0.93' ' 0.44 - 0.50' ' 0.69 - 0.85'
 ' 0.62 - 0.80' ' 0.13 - 0.31' ' 0.68 - 1.00' ' 0.70 - 0.80' ' 0.28 - 0.62'
 ' 0.51 - 0.71' ' 0.47 - 0.49' ' 0.28 - 0.44' ' 0.33 - 0.40' ' 0.11 - 0.32'
 ' 0.29 - 0.38' ' 0.31 - 0.52' ' 0.20 - 0.41' ' 0.45 - 0.5

In [71]:
#Hop bitterness: very low (minimal), low, medium, high
#Create binary columns for each value


s = ['very low', 'minimal', 'not perceived']
df['Hopbit_minimal'] = [1 if any(x in df['Hop bitterness'][w].lower() for x in s) else 0 for w in range(len(df)) ]

df['Hopbit_low'] = [1 if 'low' in df['Hop bitterness'][w].lower() else 0 for w in range(len(df))]

m = ['medium', 'moderate']
df['Hopbit_medium'] = [1 if any(x in df['Hop bitterness'][w].lower() for x in m) else 0 for w in range(len(df))]

df['Hopbit_high'] = [1 if 'high' in df['Hop bitterness'][w].lower() else 0 for w in range(len(df))]                              


In [79]:
#Hop aroma: Citrus, Fruity, Floral, Fresh, Herbal, woody, chocolate; very low (minimal), low, medium, high

i = 'Hop Aroma/Flavor'

s = ['None,','absent', 'not perceived', 'very low', 'minimal', 'not perceived']
df['HopAF_minimal'] = [1 if any(x in df[i][w].lower() for x in s) else 0 for w in range(len(df)) ]

df['HopAF_low'] = [1 if 'low' in df[i][w].lower() else 0 for w in range(len(df))]

m = ['medium', 'moderate']
df['HopAF_medium'] = [1 if any(x in df[i][w].lower() for x in m) else 0 for w in range(len(df))]

df['HopAF_high'] = [1 if 'high' in df[i][w].lower() else 0 for w in range(len(df))]     

df['HopAF_flower'] = [1 if 'flow' in df[i][w].lower() else 0 for w in range(len(df))]

df['HopAF_herb'] = [1 if 'herb' in df[i][w].lower() else 0 for w in range(len(df))]

df['HopAF_citrus'] = [1 if 'citrus' in df[i][w].lower() else 0 for w in range(len(df))]     
df['HopAF_fruit'] = [1 if 'fruit' in df[i][w].lower() else 0 for w in range(len(df))]     
df['HopAF_choco'] = [1 if 'choco' in df[i][w].lower() else 0 for w in range(len(df))] 
df['HopAF_wood'] = [1 if 'wood' in df[i][w].lower() else 0 for w in range(len(df))] 
df['HopAF_fresh'] = [1 if 'fresh' in df[i][w].lower() else 0 for w in range(len(df))] 


